In [1]:
import random

In [2]:
class EllipticalCurve:
    """
        elliptical curve over a finite field Fp:
        y^2 = x^3 + ax + b (mod p)
    """
    
    def __init__(self, a, b, p):
        if 4*a**3 + 27*b**2 == 0:
            raise Exception(f'x^3 + {a}x + {b} is singular')
        
        self.a = a
        self.b = b
        self.p = p
        self.generators = EllipticalCurve.get_generators(a, b, p)
        self.G = self.generators[-1]
    
    @staticmethod
    def get_generators(a, b, p):
        """
            if point (h,k) lies on the y^2 = x^3 + ax + b (mod p)
            then,
            (h^3 + ah + b - k^2) % p = 0
        """
        
        generators = []
        for x in range(p):
            for y in range(p):
                if (x**3 + a*x + b - y**2) % p == 0:
                    generators.append((x,y))
        
        return generators
    
    def mod(self, e):
        return e%self.p
    
    def invmod(self, e):
        return pow(e, self.p-2, self.p)
    
    def add(self, P1, P2):
        """
            for tangent:
                u = (3x1^2 + a) * invmod(2y1) (mod p)
            for two points:
                u = (y2-y1) * invmod(x2-x1) (mod p)
                
            xr = u^2 - x1 - x2 (mod p)
            yr = u(x1 - xr) - y1 (mod p)
        """
        x1, y1 = P1
        x2, y2 = P2
        u = 0
        
        if self.mod(x2 - x1) == 0 and self.mod(y2 - y1) == 0:
            u = self.mod((3*x1*x1 + self.a) * self.invmod(2*y1))
        else:
            u = self.mod((y2-y1) * self.invmod(x2-x1))
            
        xr = self.mod(u*u - x1 - x2)
        yr = self.mod(u*(x1-xr) - y1)
        return (xr, yr)
    
    def sub(self, P1, P2):
        x2, y2 = P2
        P2 = (x2, -y2)
        return self.add(P1, P2)
    
    def mul(self, n, P):
        res = P
        while n > 1:
            res = self.add(res, P)
            n = n-1
        return res

In [3]:
# curve: p'1707'
curve = EllipticalCurve(a=0, b=7, p=17)

In [4]:
print('possible generator points:\n', curve.generators)

possible generator points:
 [(1, 5), (1, 12), (2, 7), (2, 10), (3, 0), (5, 8), (5, 9), (6, 6), (6, 11), (8, 3), (8, 14), (10, 2), (10, 15), (12, 1), (12, 16), (15, 4), (15, 13)]


In [5]:
G = curve.G
print('generator point (G):', G)

generator point (G): (15, 13)


In [6]:
# private key for A
ka = 13

# pubic key for A
# Pa = ka.G
Pa = curve.mul(ka, G)
print('public key for A:', Pa)

public key for A: (6, 11)


In [7]:
# private key for B
kb = 29

# public key for B
# Pb = kb.G
Pb = curve.mul(kb, G)
print('public key for B:', Pb)

public key for B: (15, 4)


In [8]:
# plain message (M)
M = (3,4)
print('plain message:',M)

plain message: (3, 4)


### Encrypting with Pb

In [9]:
# random +ve number (r)
r = random.randint(1,100)

In [10]:
# C1 = r.G
C1 = curve.mul(r, G)

# C2 = M + rPb
rPb = curve.mul(r, Pb)
C2 = curve.add(M, rPb)

# cipher (C) = [C1, C2]
C = [C1, C2]
print('cipher:', C)

cipher: [(15, 4), (7, 10)]


### Decrypting with kb

In [11]:
# M = C2 - kbC1

kbC1 = curve.mul(kb, C1)
M = curve.sub(C2, kbC1)
print('decrypted message:', M)

decrypted message: (3, 4)
